In [24]:
import sys
import os
import py_entitymatching as em
print('magellan version:' + em.__version__)
import re
import csv
import pandas as pd

magellan version:0.1.0


In [25]:
working_dir = os.path.dirname(os.getcwd())
path_to_csv_dir = working_dir + os.sep + 'csv_files'+ os.sep
path_to_csv_dir

'/Users/carepjan/code/website/stage3/csv_files/'

# STEP 1 - PRE-PROCESSING DATA

In this stage, we need to preprocess data before applying Megellan. This is because our datasets (especially the AOM dataset) are quite dirty, and therefore adversely affecting Megellan's blocking and matching functions. For example, states can take any value of "CA", "California", or "CA - California".

In this step, we will clean the following variables:
* Country name (e.g. Whed data has 2 Belgiums: (1) Belgium - French Community and (2) Belgium - Flemish Community)
* State name
* City name
* Affiliation name
* Email server domain (we will only capture the university information from the email server domain - if there is any)


In [26]:
country_domains = ["ac","ad","ae","af","ag","ai","al","am","an","ap","aq","ar","as","at","au",
                  "az","ba","bb","be","bf","bg","bh","bi","bm","bn","bo","br","bt","by","bz",
                  "ca","cc","cd","cf","cg","ch","ck","cl","cm","cn","co","cr","cu","cx","cy",
                  "cz","de","dj","dk","do","dz","ec","ee","eg","es","fi","fj","fk","fm","fr",
                  "fo","gb","ge","gf","gg","gh","gi","gl","gm","gn","gr","gs","gt","gu","hk",
                  "hm","hn","hr","hu","id","ie","il","im","in","int","io","ir","is","it","je",
                  "jo","jp","ke","kg","kh","kr","kw","ky","kz","lb","lc","li","lk","lr","lt",
                  "lu","lv","ly","mc","md","mg","mh","mk","mm","mn","mo","mp","mq","mr","ms",
                  "mt","mu","mx","my","mw","na","nc","nf","ni","nl","no","np","nu","nz","om",
                  "pa","pe","pg","ph","pk","qa","re","ro","ru","rw","sa","sb","se","sg","sh",
                  "si","sk","sm","sn","so","st","su","sv","sz","tc","td","tf","th","tj","tm",
                  "tn","to","tp","tr","tt","tv","tw","tz","ua","ug","uk","um","us","uy","uz",
                  "ve","vg","vi","vu","wf","ws","yt","yu","za","zm"]
academic_domains = ["edu", "ac"]
common_domains = ["net","com","info","org"]
country_dict = {        
    "Belgium - Flemish Community": "belgium",
    "Belgium - French Community": "belgium",
    "Bolivia (Plurinational State of)": "bolivia",
    "China - Hong Kong SAR": "hong kong",
    "China - Macao SAR": "macau",
    "China - Taiwan": "taiwan",
    "Congo (Democratic Republic)": "congo",
    "France - French Guyana": "france",
    "France - French Polynesia": "france",
    "France - Guadeloupe": "france",
    "France - Martinique": "france",
    "France - New Caledonia": "france",
    "France - Reunion": "france",
    "Gambia (The)": "gambia",
    "Iran (Islamic Republic of)": "iran",
    "Korea (Democratic People's Republic of)": "north korea",
    "KOREA, REPUBLIC OF": "south korea",
    "Lao People's Democratic Republic": "laos",
    "Macedonia (The Former Yugoslav Republic)": "macedonia",
    "TRINIDAD": "trinidad and tobago",
    "United States of America ": "united states",
    "Venezuela (Bolivarian Republic of)": "venezuela",
    "Korea (Republic of)": "south korea"
}  

us_states = {
  "AB": { "country": "canada", "province": "Alberta" },
  "AK": "Alaska",
  "AL": "Alabama",
  "AR": "Arkansas",
  "AZ": "Arizona",
  "Baltimore": "Maryland",
  "Brighton": "Colorado",
  "CA": "California",
  "CA - California": "California",
  "California(CA)": "California",
  "CO": "Colorado",
  "CT": "Connecticut",
  "D.C.": "District of Columbia",
  "DC": "District of Columbia",
  "DE": "Delaware",
  "District of Col": "District of Columbia",
  "FL": "Florida",
  "GA": "Georgia",
  "HI": "Hawaii",
  "IA": "Iowa",
  "ID": "Idaho",
  "IL": "Illinois",
  "IN": "Indiana",
  "KS": "Kansas",
  "KY": "Kentucky",
  "LA": "Louisiana",
  "Lisbon": { "country": "portugal", "province": "Lisbon"},
  "M0": "Missouri",
  "MA": "Massachusetts",
  "MA - Massachuse": "Massachusetts",
  "MA.": "Massachusetts",
  "Mass.": "Massachusetts",
  "MD": "Maryland",
  "MD - Maryland": "Maryland",
  "ME": "Maine",
  "MI": "Michigan",
  "MI.": "Michigan",
  "Minn": "Minnesota",
  "Minnesota": "Minnesota",
  "MN": "Minnesota",
  "MO": "Missouri",
  "MS": "Mississippi",
  "MT": "Montana",
  "NC": "North Carolina",
  "ND": "North Dakota",
  "NE": "Nebraska",
  "NH": "New Hampshire",
  "NJ": "New Jersey",
  "NM": "New Mexico",
  "NV": "Nevada",
  "NY": "New York",
  "OH": "Ohio",
  "OK": "Oklahoma",
  "OR": "Oregon",
  "PA": "Pennsylvania",
  "Penn": "Pennsylvania",
  "RI": "Rhode Island",
  "SC": "South Carolina",
  "SD": "South Dakota",
  "TN": "Tennessee",
  "TX": "Texas",
  "TX - Texas": "Texas",
  "UT": "Utah",
  "VA": "Virginia",
  "VT": "Vermont",
  "WA": "Washington",
  "WI": "Wisconsin",
  "wisconsin": "Wisconsin",
  "WV": "West Virginia",
  "WY": "Wyoming"
}

### 1.A. Clean AOM data

In [38]:
data = pd.read_csv(path_to_csv_dir + '_aom.csv', encoding = 'UTF-8', index_col = 'person_id')
for i in ['full_name','first_name','last_name','phone','fax','email','person_html_name']:
    del data[i]
data.head(n=3)

### CLEAN AFFILIATION INFORMATION
def aomclean_affiliation(a_name):
    a_name = a_name.str.replace('U.','University')

    to_remove = [' of ', ' at ', ' in ',',', '-', '&', '(', ')', r'\s+']
    for i in to_remove:
        a_name = a_name.str.replace(i, " ")

    a_name = a_name.str.replace("'",'')
    a_name = a_name.str.lower()
    return a_name

### CLEAN EMAIL SERVER INFORMATION
def aomclean_email_server(a_email_server):
    # This function aims to return the essential/ importantant information from individuals' email
    # server domain. For example, in the case of email server of "wharton.upenn.edu" and university 
    # website address of "http://www.upenn.edu", important is the information "upenn". This is
    # because we want to map affiliations at the university level, instead of the school level,
    #  e.g. Wharton is the business school of University of Pensylvenia. Subsequently, a feature will
    #  capture whether the essential information is contained in the affiliation's website address

    # In order to extract the important information, we eliminate (1) country domains (e.g. "au" is
    # Australia's country domain), (2) academic domains (i.e. "edu" and "ac" )), and (3) common domains
    # (e.g. "com, "net", "info"). After eliminating these domains, we take the first info - which
    # is supposed to capture the highest level of domain at the affiliation level. For example, in
    # the case of "wharton.upenn.edu", after eliminating the abovementioned domains, we will have
    # "wharton.upenn", of which "upenn" is at the university level and "wharton" is at the school
    # level. We will only capture the university-evel info, i.e. "upenn"


    # EXAMPLE: 
    # "Catolica Lisbon School of Business and Economics" - is the business school of Catholic U. Portugal
    # AOM's email server domain: "clsbe.lisboa.ucp.pt" ; WHED's affiliation website: "http://www.ucp.pt"
    # -----
    # The essential information of the email server domain is "ucp"
    # "ucp" appears in the affiation website
    # --> This helps link school to university !!!!!!!! YAYYYYYYY
    global country_domain 
    global academic_domains 
    global common_domains

    n = a_email_server.size
    for i in range (0, n):
        email_server = a_email_server.iloc[i].strip().split('.')

        # The sequence of the following conditional commands is important. It is because the country
        # domains are usually the last one on the right of an email server domain, and it is followed
        # either by academic domain and/or common domain. 
        if email_server[-1] in country_domains:
            del email_server[len(email_server)-1]

        if email_server[-1] in academic_domains:
            del email_server[len(email_server)-1]
        if email_server[-1] in common_domains:
            del email_server[len(email_server)-1]
        if email_server[-1] in academic_domains:
            del email_server[len(email_server)-1]

        # return the highest level information only
        a_email_server.iloc[i] = email_server[len(email_server)-1]
    return a_email_server

### CLEAN CITY INFORMATION
def aomclean_city(a_city):
    df = pd.read_csv(path_to_csv_dir + "aom_list.csv")
    inputs = df['Input'].values
    outputs = df['Output'].values
    D = dict(zip(inputs,outputs))

    n = a_city.size
    for i in range (0, n):
        if a_city.iloc[i] in D:
            a_city.iloc[i] = D[a_city.iloc[i]]
            
    to_remove = [' of ', ' at ', ' in ',',','.', '-', '&', '(', ')',r'\s+' ]
    for i in to_remove:
        a_city = a_city.str.replace(i, " ")
    a_city = a_city.str.replace("'",'')
    a_city = a_city.str.lower()        
            
    return a_city

### CLEAN COUNTRY INFORMATION
def aomclean_country(a_country):
    global country_dict
    
    n = a_country.size
    for i in range (0, n):
        if a_country.iloc[i] in country_dict:
            a_country.iloc[i] = country_dict[a_country.iloc[i]]
    a_country = a_country.str.lower()
    a_country = a_country.str.strip()
    return a_country

### CLEAN STATES INFORMATION
def aomclean_states(a_prov, a_country):
    global us_states
    
    n = a_prov.size
    for i in range (0, n):
        if a_prov.iloc[i] in us_states:
            if isinstance(us_states[a_prov.iloc[i]], str):
                a_prov.iloc[i] = us_states[a_prov.iloc[i]]
            else:
                a_country.iloc[i] = us_states[a_prov.iloc[i]]["country"]
                a_prov.iloc[i] = us_states[a_prov.iloc[i]]["province"]                
    a_prov = a_prov.str.lower()
    a_prov = a_prov.str.strip()
    return (a_prov, a_country)

### --------------------------------------------------------
### CLEANING
### --------------------------------------------------------
data.a_name = aomclean_affiliation(data.a_name)
data.a_email_server = aomclean_email_server(data.a_email_server)
data.a_city = aomclean_city(data.a_city)
data.a_country = aomclean_country(data.a_country)
data.a_prov, data.a_country = aomclean_states(data.a_prov, data.a_country)

### SAVE AS A CLEANED CSV
data.to_csv(path_to_csv_dir + '_aom_cleaned.csv', encoding = 'UTF-8', index_label = 'person_id')
data.head(n=3)
# data.a_country.value_counts()

,a_name,a_city,a_prov,a_country,a_email_server
person_id,,,,,
1,academy management,briarcliff manor,new york,united states,aom
4,northeastern university,boston,massachusetts,united states,gmail
5,skidmore college,saratoga springs,new york,united states,skidmore


### 1.B. Clean WHED data

In [48]:
data = pd.read_csv(path_to_csv_dir + '_whed.csv', encoding = 'ISO-8859-1', index_col = 'a_id')
for i in ['a_street','a_pcode','a_tel']:
    del data[i]
data.head(n=3)

### CLEAN AFFILIATION INFORMATION
def whedclean_affiliation(a_name):
    a_name = a_name.str.replace('U.','University')

    to_remove = [' of ', ' at ', ' in ',',', '-', '&', '(', ')', r'\s+']
    for i in to_remove:
        a_name = a_name.str.replace(i, " ")

    a_name = a_name.str.replace("'",'')
    a_name = a_name.str.lower()
    return a_name

### CLEAN CITY INFORMATION
def whedclean_city(a_city):
    to_remove = [' of ', ' at ', ' in ',',','.', '-', '&', '(', ')',r'\s+' ]
    for i in to_remove:
        a_city = a_city.str.replace(i, " ")
    a_city = a_city.str.replace("'",'')
    a_city = a_city.str.lower()          
    return a_city

### CLEAN COUNTRY INFORMATION
def whedclean_country(a_country):
    global country_dict
    
    n = a_country.size
    for i in range (0, n):
        if a_country.iloc[i] in country_dict:
            a_country.iloc[i] = country_dict[a_country.iloc[i]]
    a_country = a_country.str.lower()
    a_country = a_country.str.strip()
    
    return a_country

### --------------------------------------------------------
### CLEANING
### --------------------------------------------------------
data.a_name = whedclean_affiliation(data.a_name)
data.a_city = whedclean_city(data.a_city)
data.a_country = whedclean_country(data.a_country)

### SAVE AS A CLEANED CSV
data.to_csv(path_to_csv_dir + '_whed_cleaned.csv', encoding = 'UTF-8', index_label = 'a_id')
data.head(n=3)
# data.a_country.value_counts()

,a_name,a_country,a_city,a_prov,a_web
a_id,,,,,
2,pampanga state agricultural university,philippines,magalang,Pampanga,http://www.pac.edu.ph
4,les roches international school hotel management,switzerland,bluche crans montana,Bluche-Crans-Montana,http://www.lesroches.edu
6,dharma gate budapest buddhist university,hungary,budapest,NaN,http://www.tkbf.eu


# STEP 2 - MAGELLAN - BLOCKING

In [37]:

A = em.read_csv_metadata(path_to_csv_dir + '_aom.csv', key = 'person_id')
print(em.get_key(A))

B = em.read_csv_metadata(path_to_csv_dir + '_whed.csv', key = 'a_id')
print(em.get_key(B))


Metadata file is not present in the given path; proceeding to read the csv file.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 13: invalid start byte

In [26]:
sample_A, sample_B = em.down_sample(A, B, size=500, y_param=1)


telfer school management university ottawa
